In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv
/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv


In [2]:
titles_df = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv', usecols=['id','original_title','genres','revenue','runtime',])

In [3]:
print(titles_df)

                                                 genres      id  \
0     [{"id": 28, "name": "Action"}, {"id": 12, "nam...   19995   
1     [{"id": 12, "name": "Adventure"}, {"id": 14, "...     285   
2     [{"id": 28, "name": "Action"}, {"id": 12, "nam...  206647   
3     [{"id": 28, "name": "Action"}, {"id": 80, "nam...   49026   
4     [{"id": 28, "name": "Action"}, {"id": 12, "nam...   49529   
...                                                 ...     ...   
4798  [{"id": 28, "name": "Action"}, {"id": 80, "nam...    9367   
4799  [{"id": 35, "name": "Comedy"}, {"id": 10749, "...   72766   
4800  [{"id": 35, "name": "Comedy"}, {"id": 18, "nam...  231617   
4801                                                 []  126186   
4802                [{"id": 99, "name": "Documentary"}]   25975   

                                original_title     revenue  runtime  
0                                       Avatar  2787965087    162.0  
1     Pirates of the Caribbean: At World's End   961000